# Reading Notes: Fluent Python, Chapter 1

This notebook contains notes I took from the 1st chapter of the book Fluent Python, by Luciano Ramalho. The notes are intertwined with executable code examples.

## Special Methods are your friends!

Special methods, also known as dunder methods, are predefined methods that allows us to customize the behavior of objects. They are your friends and can make for a much more Pythonic interaction with classes.

> The first thing to know about special methods is that they are meant to be called by the Python interpreter, and not by you. You don’t write `my_object.__len__()`. You write `len(my_object)` and, if my_object is an instance of a user-defined class, then Python calls the `__len__` method you implemented.

Let's implement two card deck classes, one with and the other without special methods.

In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])


class FrenchDeckWithSpecialMethods:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]


class FrenchDeckWithoutSpecialMethods:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def length(self):
        return len(self._cards)

    def get_item(self, position):
        return self._cards[position]


There are advantages to the use of special methods. Let's say you want to check the length of the deck. In Python, checking the length is usually done with the built-in `len`... However, that won't work for instances of `FrenchDeckWithoutSpecialMethods`:

In [ ]:
deck_without_special_methods = FrenchDeckWithoutSpecialMethods()

try:
  len(deck_without_special_methods)
except Exception as e:
  print(f"Something went wrong: {e}")

Something went wrong: object of type 'FrenchDeckWithoutSpecialMethods' has no len()


This happens because what `len` does is checking for the special method `__len__` of the object, then calling it. Since what the object above has instead is a regular `length` class, we would need to call `deck_without_special_methods.length()` to achieve our goal. While there is nothing inherently wrong with this, it's not intuitive to the Python programmer or Pythonic. The Pythonic way of getting the length of an object is, as said, calling the built-in `len`. Which we can do if our class leverages the special method `__len__`!

> But the interpreter takes a shortcut when dealing for built-in types (...) Python variable-sized collections written in C include a struct called `PyVarObject`, which has an `ob_size` field holding the number of items in the collection. So, if `my_object` is an instance of one of those built-ins, then `len(my_object)` retrieves the value of the `ob_size` field, and this is much faster than calling a method.

In [ ]:
deck_with_special_methods = FrenchDeckWithSpecialMethods()

print(len(deck_with_special_methods))

52


The same is true for getting an item from an object. The Intuitive way of doing that is using the `[]` operator. Like `len`, it calls a special method (`__getitem__` in this case). That means we can't use this operator in objects that have this special method. Since `deck_with_special_methods` does, it supports slicing and `in`, it is iterable, and more!

In [ ]:
print(f"First item of deck: {deck_with_special_methods[0]}")

print("First 3 cards of deck:")
for card in deck_with_special_methods[:3]:
  print(f"  {card}")

First item of deck: Card(rank='2', suit='spades')
First 3 cards of deck:
  Card(rank='2', suit='spades')
  Card(rank='3', suit='spades')
  Card(rank='4', suit='spades')


Special methods are your friends because they make for Pythonic classes, which are intuitive to use and can leverage much more of Python's built-in functionalities.

> Normally, your code should not have many direct calls to special methods. (...) If you need to invoke a special method, it is usually better to call the related built-in
function (e.g., len, iter, str, etc.).

### Uses of Special Methods

Special methods have important, more specific uses other than just making classes Pythonic (though that is often a consequence of using them).

#### Emulating Numeric Types

If a class could benefit from responding to operators like `+`, it's a good idea to have the class emulate a numeric type. A `Vector` class might come to mind.



In [2]:
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Vector({self.x!r}, {self.y!r})'

    def __abs__(self):
        return (self.x ** 2 + self.y ** 2) ** 0.5

    def __bool__(self):
        return bool(abs(self))

    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

Vectors can be added and multiplied. It makes perfect sense to then use the `__add__` and `__mul__` special methods when implementing the `Vector` class. That way, we can sum and multiply Vector objects with operators `+` and `*`:

In [ ]:
vector_1 = Vector(1, 3)
vector_2 = Vector(3, 4)
vector_3 = vector_1 + vector_2
vector_4 = vector_1 * 3

print(f"Vector 1: {vector_1}")
print(f"Vector 2: {vector_2}")
print(f"Vector 1 + Vector 2: {vector_3}")
print(f"Vector 1 * Vector 2: {vector_4}")

Vector 1: Vector(1, 3)
Vector 2: Vector(3, 4)
Vector 1 + Vector 2: Vector(4, 7)
Vector 1 * Vector 2: Vector(3, 9)


Doesn't this just *feel* better than calling a regular method to sum and store the returned value, as shown below?

```py
vector_3 = vector_1.sum(vector_2)
```

#### String Representation

In the `Vector` class, we also used special method `__repr__`. This is supposed to return the string representation of an object.

> The string returned by `__repr__` should be unambiguous and, if possible, match the source code necessary to re-create the represented object.

This method is used internally to this end. It is also called by `print` implicitly as fallback if the object has no `__str__` special method. `__str__` also serves the same purpose, but specifically exists to be called by `print`. Since sometimes the representation returned by `__repr__` is already user-friendly (`Vector` is a fitting example) and it is a fallback for `__str__` when calling `print`, the latter special method is redundant in these situations.

> Programmers with prior experience in languages with a `toString` method tend to implement `__str__` and not `__repr__`. If you only implement one of these special methods in Python, choose `__repr__`.

#### Boolean Value of a Custom Type

Like other languages, Python has the concept of non-boolean objects being *truthy* of *falsy*.



In [1]:
print(bool(1))
print(bool(0))

True


Integer values `0` are falsy and `1` are truthy. What's going on here is that the built-in `bool` is calling the special method `__bool__` of these integer objects and returning what it gets from it. This allows us to use these objects in statements like `if` and `while`, as well as operators like `and`, `or`, and `not`.

> By default, instances of user-defined classes are considered truthy, unless either `__bool__` or `__len__` is implemented. Basically, `bool(x)` calls `x.__bool__()` and uses
the result. If `__bool__` is not implemented, Python tries to invoke x.`__len__()`, and
if that returns zero, bool returns `False`. Otherwise bool returns `True`.


In the `Vector` class above, this special method is leveraged. We can therefore use it in these statements and operators.


In [3]:
truthy_vector = Vector(1, 3)
falsy_vector = Vector(0, 0)

print(f"Truthiness of Vector 1: {bool(truthy_vector)}")
print(f"Truthiness of Vector 2: {bool(falsy_vector)}")

Truthiness of Vector 1: True
Truthiness of Vector 2: False


## Conclusion
Special methods allow for Pythonic objects that behave like built-in types, giving an expressive and consistent feel to the code overall. They are your friends! Use them whenever is appropiate to make better Python code.